In [1]:
from Functions.polygon_scrape import *
from Functions.RF_Funcs import *
import pandas as pd
import numpy as np
from secret import polygon_key
from datetime import datetime, date
import polygon as pg
import warnings
warnings.filterwarnings('ignore')
import time

In [2]:
import pandas_market_calendars as mcal
nyse = mcal.get_calendar('NYSE')
start_date = '2020-01-01'
end_date = '2023-12-31'
schedule = nyse.schedule(start_date=start_date, end_date=end_date)
trading_days = schedule['market_open'].dt.date.tolist()

### Acquire Model Predictions

In [3]:
# data = pd.read_csv('Treasury.csv')

# X = data[['RV_(t-1)','RV_t Weekly', 'RV_t Monthly',
#           'r_(t-1)', 'r_t Weekly', 'r_t Monthly',
#           'VIX Close (t-1)', '1MT (t-1)']]
# Y = data['RV_t (Realized Variance)']

# X['1MT (t-1)'] = pd.to_numeric(X['1MT (t-1)'], errors='coerce')
# X['1MT (t-1)'] = X['1MT (t-1)'].fillna((X['1MT (t-1)'].shift(+1) + X['1MT (t-1)'].shift(-1)) / 2)

# op = RollingWindowRF(X,Y,data['date'])
# pred = np.sqrt(op['predictions'].loc['values'])*np.sqrt(365)

### Import data to avoid long runtime

In [3]:
pred = pd.read_csv('predictions.csv').set_index('date')['values']
print(np.average(pred))

0.13223407478542387


### Deploy the Option Strategy Using the Prediction

In [4]:
data = pd.read_csv('Treasury.csv')

Dates where Tu, Th expiry options are NOT available

In [ ]:
dates = data['date'].iloc[-250:-220]

res = []

for date in dates:
    try:
        res.append(OptionStrategy(model_estimate=pred[date],date=datetime.strptime(date,'%m/%d/%y').date(), trading_days=trading_days))
    except LookupError:
        res.append('Data Not Found')
    print(f'{date}: {res[-1]}')
    time.sleep(62)

Dates where M-F expiry options ARE available

In [5]:
dates = data['date'].iloc[-180:-150]

res = []

for date in dates:
    try:
        res.append(OptionStrategy(model_estimate=pred[date],date=datetime.strptime(date,'%m/%d/%y').date(), trading_days=trading_days))
    except LookupError:
        res.append('Data Not Found')
    print(f'{date}: {res[-1]}')
    time.sleep(62)

11/21/22: {'Profit': -2.59, 'Investment': 2.84, 'Return': '-91.39%', 'Results': {'Stock close': 394.59, 'Stock close next': 399.9, 'Call close': 1.42, 'Put close': 1.13, 'Avg IV': 0.18331678994465167}, '# Calls': 1.109, '# Puts': 1.119}
11/22/22: {'Profit': 0.62, 'Investment': 2.34, 'Return': '26.58%', 'Results': {'Stock close': 399.9, 'Stock close next': 402.42, 'Call close': 1.47, 'Put close': 1.14, 'Avg IV': 0.18354528492297828}, '# Calls': 0.709, '# Puts': 1.136}
11/23/22: {'Profit': -2.22, 'Investment': 2.22, 'Return': '-100.0%', 'Results': {'Stock close': 402.42, 'Stock close next': 402.33, 'Call close': 0.9, 'Put close': 0.86, 'Avg IV': 0.13234160530145422}, '# Calls': 1.399, '# Puts': 1.12}
11/25/22: {'Profit': -4.21, 'Investment': 2.76, 'Return': '-152.78%', 'Results': {'Stock close': 402.33, 'Stock close next': 395.91, 'Call close': 0.98, 'Put close': 1.12, 'Avg IV': 0.15277180056563877}, '# Calls': 1.506, '# Puts': 1.144}
11/28/22: {'Profit': 16.83, 'Investment': 16.83, 'Ret